In [3]:
%%capture
!pip install impedance
import importlib
import Manuel_echem_function
importlib.reload(Manuel_echem_function)
from Manuel_echem_function import *
import pandas as pd
import os, io
import requests
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sys
from nomad_api_calls import *

%matplotlib widget

import warnings
warnings.filterwarnings("ignore")

## Plotting preferences 

color_cm = ["#004f84ff", "#6cabe9ff", "#d15e57ff", "#ff7f2aff","#808080ff","#8787deff"]
# Expanded with 5 additional colors that harmonize with the original palette
color_cm = color_cm + ["#00a591ff", "#b2df8aff", "#f9c74fff", "#178eaeff", "#a70d88ff", "#373438ff",]


matplotlib.rc('font', size= 14) # controls default text sizes
matplotlib.rc('axes', titlesize=14) # fontsize of the axes title
matplotlib.rc('axes', labelsize=14) # fontsize of the x and y labels
matplotlib.rc('xtick', labelsize=14) # fontsize of the tick labels
matplotlib.rc('ytick', labelsize=14) # fontsize of the tick labels
matplotlib.rc('legend', fontsize= 10) # legend fontsize
matplotlib.rc('figure', titlesize=14) # fontsize of the figure title



In [4]:
url = "https://nomad-hzb-se.de/nomad-oasis/api/v1"
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN'] 
if len(token) > 10:
    print("Token Recieved")

Token Recieved


In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

from IPython.display import Markdown, display

display(Markdown("""
<h1 style='text-align:center; font-weight:bold;'>
Electrochemical Analysis
</h1>
<h3 style='text-align:center; color:gray;'>
Voilá Interactive Interface
</h3>
"""))

# --------------------------------------------------------
# ITO reference input
# --------------------------------------------------------
ito_box = widgets.Text(
    description="ITO ref:",
    placeholder="Enter ITO sample IDs, comma-separated"
)
display(ito_box)

# --------------------------------------------------------
# UI: number of experiments
# --------------------------------------------------------
num_experiments_widget = widgets.IntSlider(
    value=3, min=1, max=20, step=1,
    description="Experiments:"
)
display(num_experiments_widget)

# Container for experiment inputs
experiments_container = widgets.VBox()
display(experiments_container)

# --------------------------------------------------------
# Create one textbox per experiment
# --------------------------------------------------------
def build_experiment_inputs(change):
    boxes = []
    for i in range(change['new']):
        tb = widgets.Text(
            description=f"Exp {i+1}:",
            placeholder="Sample IDs, comma-separated"
        )
        boxes.append(tb)
    experiments_container.children = boxes

num_experiments_widget.observe(build_experiment_inputs, names='value')
build_experiment_inputs({'new': num_experiments_widget.value})


# --------------------------------------------------------
# Axis limits for EIS
# --------------------------------------------------------
xlim_min = widgets.FloatText(value=-10, description="x min:")
xlim_max = widgets.FloatText(value=3000, description="x max:")
ylim_min = widgets.FloatText(value=-10, description="y min:")
ylim_max = widgets.FloatText(value=3000, description="y max:")

limits_box = widgets.HBox([xlim_min, xlim_max, ylim_min, ylim_max])
display(limits_box)


# --------------------------------------------------------
# Button to plot EIS
# --------------------------------------------------------
plot_eis_button = widgets.Button(
    description="Plot EIS",
    button_style="info"
)
eis_output = widgets.Output()
display(plot_eis_button, eis_output)


# --------------------------------------------------------
# Internal file_id builder
# --------------------------------------------------------
def build_file_id_internal():

    file_id = []

    # 1) ITO row
    if ito_box.value.strip() == "":
        file_id.append([])
    else:
        row = [s.strip() for s in ito_box.value.split(",") if s.strip() != ""]
        file_id.append(row)

    # 2) Experiments
    for tb in experiments_container.children:
        if tb.value.strip() == "":
            file_id.append([])
        else:
            row = [s.strip() for s in tb.value.split(",") if s.strip() != ""]
            file_id.append(row)

    return file_id


# --------------------------------------------------------
# EIS PLOT FUNCTION
# --------------------------------------------------------
def plot_eis(button):
    with eis_output:
        clear_output()
        
        # Build file_id internally
        file_id = build_file_id_internal()

        fig, axis = plt.subplots(1, 1, figsize=(7, 7))
        axs = axis
        color_cm = plt.cm.tab10

        for jj, subbatch in enumerate(file_id):

            color = color_cm(jj % 10)

            for ii, file in enumerate(subbatch):

                # --------------------------
                # Spin coating info
                # --------------------------
                try:
                    info = get_specific_data_of_sample(url, token, file, "HySprint_SpinCoating")
                except AssertionError:
                    print(f"⚠ Sample ID '{file}' not found. Skipping.")
                    continue

                # Label only for first repetition
                if ii==0:            
                    #Get Concentration and SAM information from NOMAD
                    A=get_sample_description(url, token, [file])
                    label=A[file]        
                else: 
                    label=None

                # --------------------------
                # EIS data
                # --------------------------
                try:
                    data = get_specific_data_of_sample(url, token, file, "HySprint_ElectrochemicalImpedanceSpectroscopy")
                except AssertionError:
                    print(f"⚠ No EIS data for '{file}'. Skipping.")
                    continue

                axs.plot(data[0]["z_real"], data[0]["z_imaginary"], "o",
                         color=color, label=label)

        # Formatting
        axs.set_aspect("equal")
        axs.set_xlabel(r"$Z'$ $(\Omega)$")
        axs.set_ylabel(r"$Z''$ $(\Omega)$")

        axs.set_xlim([xlim_min.value, xlim_max.value])
        axs.set_ylim([ylim_min.value, ylim_max.value])

        axs.legend()
        plt.show()


plot_eis_button.on_click(plot_eis)

# --------------------------------------------------------
# Plot DPV button
# --------------------------------------------------------
plot_dpv_button = widgets.Button(
    description="Plot DPV",
    button_style="success"
)
dpv_output = widgets.Output()
display(plot_dpv_button, dpv_output)


def plot_dpv(button):
    with dpv_output:
        clear_output()

        file_id = build_file_id_internal()
        color_cm = plt.cm.tab10

        fig, ax = plt.subplots(figsize=(7, 7))

        for jj, subbatch in enumerate(file_id):

            color = color_cm(jj % 10)

            for ii, file in enumerate(subbatch):

                # Label logic (same as EIS)
                try:
                    info = get_specific_data_of_sample(url, token, file, "HySprint_SpinCoating")
                except AssertionError:
                    print(f"⚠ Sample '{file}' not found. Skipping.")
                    continue


                # Label only for first repetition
                if ii==0:            
                    #Get Concentration and SAM information from NOMAD
                    A=get_sample_description(url, token, [file])
                    label=A[file]
        
                else: 
                    label=None
                    
                # DPV data
                try:
                    dpv = get_specific_data_of_sample(url, token, file, "HySprint_DifferentialPulseVoltammetry")
                except AssertionError:
                    print(f"⚠ No DPV data for '{file}'. Skipping.")
                    continue

                ax.plot(dpv[0]["voltage"], dpv[0]["current"],
                        color=color, label=label)

        ax.set_xlabel("Voltage (V)")
        ax.set_ylabel("Differential current (A)")
        ax.legend()
        plt.show()

plot_dpv_button.on_click(plot_dpv)
# --------------------------------------------------------
# CV Parameter Inputs
# --------------------------------------------------------

reference_values_box = widgets.Text(
    description="ref. speeds:",
    value="0.05, 0.1, 0.2, 0.3, 0.5",
    placeholder="Comma-separated scan speeds"
)

threshold_box = widgets.FloatText(
    description="Threshold:",
    value=0.02
)

scan_speed_box = widgets.FloatText(
    description="scan speed:",
    value=0.05
)

cycle_number_box = widgets.IntText(
    description="Cycle #:",
    value=1
)

cv_param_box = widgets.VBox([
    reference_values_box,
    threshold_box,
    scan_speed_box,
    cycle_number_box
])

display(cv_param_box)

# --------------------------------------------------------
# Plot CV button
# --------------------------------------------------------
plot_cv_button = widgets.Button(
    description="Plot CV",
    button_style="warning"
)

cv_output = widgets.Output()
display(plot_cv_button, cv_output)


def plot_cv(button):
    with cv_output:
        clear_output()

        # --------------------------------------
        # Build file_id internally
        # --------------------------------------
        file_id = build_file_id_internal()
        color_cm = plt.cm.tab10
        
        # --------------------------------------
        # Parse CV parameters from widget input
        # --------------------------------------
        reference_values = [float(s.strip()) for s in reference_values_box.value.split(",")]
        threshold = threshold_box.value
        scan_speed = scan_speed_box.value
        cycle_number = cycle_number_box.value
        
        # error check
        if scan_speed not in reference_values:
            print(f"⚠ scan_speed={scan_speed} must be one of reference_values={reference_values}")
            return
        
        # --------------------------------------
        # Make figure
        # --------------------------------------
        fig, ax = plt.subplots(figsize=(7, 7))
        
        # --------------------------------------
        # CV loop
        # --------------------------------------
        for jj, subbatch in enumerate(file_id):

            color = color_cm(jj % 10)

            for ii, file in enumerate(subbatch):

                # ------------------------------
                # SpinCoating information
                # ------------------------------
                try:
                    info = get_specific_data_of_sample(url, token, file, "HySprint_SpinCoating")
                except AssertionError:
                    print(f"⚠ Sample '{file}' not found.")
                    continue


                # Label only for first repetition
                if ii==0:            
                    #Get Concentration and SAM information from NOMAD
                    A=get_sample_description(url, token, [file])
                    label=A[file]
        
                else: 
                    label=None

                # ------------------------------
                # CV data get
                # ------------------------------
                try:
                    data = get_specific_data_of_sample(url, token, file, "HySprint_CyclicVoltammetry")
                except AssertionError:
                    print(f"⚠ No CV data for '{file}'. Skipping.")
                    continue

                # Extract full cycle dataframe
                CV_df = CV_analysis_NOMAD(
                    data[0]["cycles"][0]["time"],
                    data[0]["cycles"][0]["voltage"],
                    data[0]["cycles"][0]["current"],
                    reference_values,
                    threshold,
                    plot_SS=False
                )

                # Filter by scan speed bin
                df1 = CV_df[CV_df["Speed_bin"] == f"{scan_speed:.2f}"]

                # Assign cycle numbers
                df1 = assign_cycles_NOMAD(df1)

                # Extract selected cycle
                df_cycle = df1[df1["Cycle"] == cycle_number]

                # Skip if empty
                if df_cycle.empty:
                    print(f"⚠ No CV data for cycle {cycle_number} in '{file}'.")
                    continue

                # Plot CV curve
                ax.plot(df_cycle["voltage"], df_cycle["current"] * 1e3,
                        color=color, label=label)

        # --------------------------------------
        # Format plot
        # --------------------------------------
        ax.set_xlabel("Voltage (V)")
        ax.set_ylabel("Current (mA)")
        ax.legend()
        plt.show()


plot_cv_button.on_click(plot_cv)




<h1 style='text-align:center; font-weight:bold;'>
Electrochemical Analysis
</h1>
<h3 style='text-align:center; color:gray;'>
Voilá Interactive Interface
</h3>


Text(value='', description='ITO ref:', placeholder='Enter ITO sample IDs, comma-separated')

IntSlider(value=3, description='Experiments:', max=20, min=1)

VBox()

Button(button_style='info', description='Plot EIS', style=ButtonStyle())

Output()

Button(button_style='success', description='Plot DPV', style=ButtonStyle())

Output()

Button(button_style='warning', description='Plot CV', style=ButtonStyle())

Output()